<a href="https://colab.research.google.com/github/41371232H/PL_Repo/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW6
### pdf檔案可以轉成CSV檔並存進googlesheet中，可以搜尋本周課表，或是以星期搜尋，可選擇要修改的課程進行攜帶品、先讀章節和備註的修改並寫回googlesheet，並且可以下載當周的課表CSV檔，或是選擇將CSV檔傳到gmail上。
googlesheet連結:https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit?gid=0#gid=0

# 最終程式碼

In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import io
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 保留所有欄位
    return week_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"

    parts = selected_row.split("\t")
    if len(parts) < 6:
        return "❌ 無法解析選擇的行"

    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    # 找到目標行
    mask = (
        (df["日期"] == pd.to_datetime(date).date()) &
        (df["星期"] == weekday) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start_time) &
        (df["時間(迄)"] == end_time) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程，請檢查 Google Sheet 資料是否格式一致"

    # Google Sheet 第幾列（+2 因為 index 從 0 且第 1 列是標題）
    row_index = df.index[mask][0] + 2

    # 更新 Google Sheet（安全版，只動該行欄位）
    ws.update_cell(row_index, df.columns.get_loc("攜帶品") + 1, carry_item)
    ws.update_cell(row_index, df.columns.get_loc("先讀章節") + 1, pre_read)
    ws.update_cell(row_index, df.columns.get_loc("備註") + 1, note)

    return "✅ 更新成功（你的資料不會被刪除）！"


# 匯出指定週次 CSV（目前先只實作「本週」）
def export_week_csv(week_option):
    week_df, _ = load_week_schedule()

    file_path = "/content/本週課表.csv"
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    return file_path

def export_and_send_week_csv(recipient_email):
    # 1️⃣ 先生成 CSV
    file_path = "/content/本週課表.csv"
    week_df, _ = load_week_schedule()
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    # 2️⃣ 寄信
    sender_email = "41371232h@gapps.ntnu.edu.tw"
    app_password = "s****************l"  #有個人密碼，先隱藏起來

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = "本週課表 CSV"
    msg.attach(MIMEText("請查收本週課表 CSV", "plain"))

    with open(file_path, "rb") as f:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(f.read())
        encoders.encode_base64(part)

        # 中文檔名正確
        from email.utils import encode_rfc2231
        filename = "本週課表.csv"
        filename_encoded = encode_rfc2231(filename, 'utf-8')
        part.add_header(
            "Content-Disposition",
            f"attachment; filename*=UTF-8''{filename_encoded}"
        )
        msg.attach(part)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)

    return "✅ CSV 已成功寄到 Gmail！"
# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

        with gr.Tab("④ 匯出 CSV"):
            week_dropdown = gr.Dropdown(choices=["本週"], label="選擇週次")
            download_btn = gr.File(label="下載 CSV")
            gr.Button("匯出 CSV").click(export_week_csv, inputs=week_dropdown, outputs=download_btn)

            # ---- 新增按鈕：寄送 CSV 到 Gmail ----
            recipient_input = gr.Textbox(label="收件人 Gmail", placeholder="你的郵件")
            send_btn = gr.Button("寄送 CSV 到 Gmail")
            send_out = gr.Textbox(label="寄送結果")

            # 寄送按鈕：輸入 CSV 路徑 + 收件人
            # 這裡固定 CSV 路徑和 export_week_csv 產生的檔案一致
            send_btn.click(
                fn=export_and_send_week_csv,
                inputs=recipient_input,
                outputs=send_out
            )

    return demo

demo = main_app()
demo.launch(inbrowser=True)

# 歷程記錄

In [ ]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import files
import os

print("請上傳您的 PDF 課表檔案...")
# 顯示上傳按鈕
uploaded = files.upload()

if not uploaded:
  print("\n⚠️ 您沒有上傳任何檔案。")
else:
  # 取得上傳檔案的名稱
  pdf_filename = next(iter(uploaded))
  print(f"\n✅ 成功上傳檔案: '{pdf_filename}'")

  # 檢查是否為 PDF (簡易檢查)
  if not pdf_filename.lower().endswith('.pdf'):
    print(f"⚠️ 警告：您上傳的檔案 '{pdf_filename}' 不是 PDF 檔。")

請上傳您的 PDF 課表檔案...


Saving export.pdf to export.pdf

✅ 成功上傳檔案: 'export.pdf'


In [ ]:
%store pdf_filename

Stored 'pdf_filename' (str)


In [ ]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

In [ ]:
import time

# 讀取上一個儲存格存的檔案名稱
try:
    %store -r pdf_filename
    if not os.path.exists(pdf_filename):
        print(f"❌ 找不到檔案 '{pdf_filename}'，請重新執行上一個儲存格上傳。")
    else:
        print(f"準備處理檔案: '{pdf_filename}'")

        # 1. 上傳檔案到 Gemini File API
        print("正在上傳 PDF 至 Gemini File API...")
        gemini_file = genai.upload_file(path=pdf_filename,
                                        display_name="114-1 課程表 PDF")
        print(f"✅ 檔案上傳成功！(File URI: {gemini_file.uri})")

        # 2. 準備 Prompt
        # 這就是您第一次給我的指示，我把它變成了給 Gemini 的 Prompt
        prompt_text = f"""
        你是一位 helpful AI assistant。
        請分析我上傳的 PDF 課表檔案 (檔案名稱: {pdf_filename})。

        這份 PDF 課表是「國立臺灣師範大學114學年度第1學期課表」。
        授課教師是「蔡芸琤」。

        請幫我將這份課表，轉換成每周的紀錄。
        學期開學日是 2025/9/1，學期結束日是 2025/12/19。

        你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，
        不要有任何其他說明文字 (例如 "好的，這是..." 或 "```csv" 標記)。

        CSV 格式的欄位必須如下：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"

        - 「攜帶品」、「先讀章節」、「備註」這三欄因為 PDF 中沒有，請保持空白。
        - 「日期」範圍請涵蓋 2025/9/1 到 2025/12/19 之間所有有課的日期。
        - 請仔細解析 PDF 中的表格，找出所有課程、時間和地點。
        """

        # 3. 呼叫 Gemini 1.5 Pro 模型
        print("正在向 Gemini 1.5 Pro 提出請求 (這可能需要一點時間)...")
        model = genai.GenerativeModel('gemini-2.5-pro')

        # 發送包含「Prompt文字」和「上傳的檔案」的請求
        response = model.generate_content([prompt_text, gemini_file])

        # 4. 刪除在 Gemini 雲端上的檔案 (避免佔用空間)
        print("正在從 Gemini 雲端刪除暫存檔案...")
        genai.delete_file(gemini_file.name)
        print("✅ 暫存檔案已刪除。")

        # 5. 顯示結果並儲存
        csv_output = response.text.strip().replace("```csv\n", "").replace("```", "")
        print("\n--- 🤖 Gemini 產生的 CSV 內容 ---")
        print(csv_output)
        print("---------------------------------")

        # 6. 將 Gemini 產生的 CSV 內容存成一個檔案
        with open("gemini_schedule.csv", "w", encoding='utf-8') as f:
            f.write(csv_output)

        print("\n✅ 已將 Gemini 產生的結果儲存為 'gemini_schedule.csv'")

except NameError:
    print("❌ 找不到 'pdf_filename'。請先執行上一個儲存格來上傳檔案。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")
    # 如果發生錯誤，也嘗試刪除已上傳的檔案
    try:
        if 'gemini_file' in locals():
            genai.delete_file(gemini_file.name)
            print("已清理遠端暫存檔案。")
    except Exception as del_e:
        print(f"清理檔案時發生錯誤: {del_e}")

準備處理檔案: 'export.pdf'
正在上傳 PDF 至 Gemini File API...
✅ 檔案上傳成功！(File URI: https://generativelanguage.googleapis.com/v1beta/files/1v4x33kjnyl0)
正在向 Gemini 1.5 Pro 提出請求 (這可能需要一點時間)...
正在從 Gemini 雲端刪除暫存檔案...
✅ 暫存檔案已刪除。

--- 🤖 Gemini 產生的 CSV 內容 ---
"日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
"2025/09/01","一","線性代數","14:20","15:10","E 102","","",""
"2025/09/02","二","網際網路概論","09:10","12:10","科技系TB311教室","","",""
"2025/09/02","二","管理學入門","13:20","15:10","誠102","","",""
"2025/09/03","三","試算表進階應用與程式設計","10:20","12:10","教402","","",""
"2025/09/03","三","體育(體操遊戲初級)","13:20","15:10","體操房","","",""
"2025/09/04","四","程式語言","09:10","12:10","科技系TB311教室","","",""
"2025/09/04","四","英文(三):科技英文","13:20","15:10","正404","","",""
"2025/09/05","五","線性代數","10:20","12:10","E 102","","",""
"2025/09/05","五","電路學","14:20","17:20","教103","","",""
"2025/09/05","五","作業系統","17:30","20:25","科技系TA509教室","","",""
"2025/09/08","一","線性代數","14:20","15:10","E 102","","",""
"2025/09/09","二","網際網路概論","09:10","12:10","科

In [ ]:
import pandas as pd
import gspread

# --- 1. 定義您的 Google Sheet 資訊 (您已提供) ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"
# --- 2. 讀取 Gemini 產生的 CSV 檔案 ---
# (假設 'gemini_schedule.csv' 檔案在 Colab 的根目錄下)
try:
    df_to_upload = pd.read_csv("gemini_schedule.csv")
    print(f"✅ 成功讀取 'gemini_schedule.csv'，共 {len(df_to_upload)} 筆資料。")

    # --- 3. 準備 gspread (假設 gc 已經過授權) ---
    # gc = gspread.authorize(creds) # 這行在您之前的儲存格應該已經執行過了
    df_to_upload = df_to_upload.fillna('')

    # --- 4. 開啟 Google Sheet 和指定的分頁 ---
    print(f"正在開啟 Google Sheet (URL: {SHEET_URL})...")
    gsheets = gc.open_by_url(SHEET_URL)

    print(f"正在存取分頁: '{WORKSHEET_NAME}'...")
    try:
        # 嘗試取得分頁，如果不存在就建立一個新的
        worksheet = gsheets.worksheet(WORKSHEET_NAME)
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到分頁 '{WORKSHEET_NAME}'，將自動建立新分頁...")
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    # --- 5. 將 DataFrame 轉換為 list (包含標頭) ---
    data_to_write = [df_to_upload.columns.values.tolist()] + df_to_upload.values.tolist()

    # --- 6. 清空分頁並寫入新資料 ---
    print("正在清空分頁內容...")
    worksheet.clear()

    print(f"正在將 {len(data_to_write) - 1} 筆資料寫入 '{WORKSHEET_NAME}' 分頁...")
    worksheet.update(data_to_write)

    print(f"\n✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '{WORKSHEET_NAME}' 分頁！")

except FileNotFoundError:
    print("❌ 錯誤：找不到 'gemini_schedule.csv' 檔案。")
    print("請確認您執行 Gemini API 的儲存格已成功產生這個檔案。")
except pd.errors.EmptyDataError:
    print("❌ 錯誤：'gemini_schedule.csv' 是空的，沒有資料可以上傳。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")

✅ 成功讀取 'gemini_schedule.csv'，共 160 筆資料。
正在開啟 Google Sheet (URL: https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit?gid=0#gid=0)...
正在存取分頁: '工作表1'...
正在清空分頁內容...
正在將 160 筆資料寫入 '工作表1' 分頁...

✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '工作表1' 分頁！


In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,日期,星期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
0,2025/09/01,一,線性代數,14:20,15:10,E 102,,,
1,2025/09/02,二,網際網路概論,09:10,12:10,科技系TB311教室,,,
2,2025/09/02,二,管理學入門,13:20,15:10,誠102,,,
3,2025/09/03,三,試算表進階應用與程式設計,10:20,12:10,教402,,,
4,2025/09/03,三,體育(體操遊戲初級),13:20,15:10,體操房,,,


In [ ]:
import pytz
from IPython.display import display, Markdown
import datetime as dt

# === 參數：指定要查的星期 ===
# 可填 "一","二","三","四","五","六","日"，或用英文 "Mon" ... "Sun"
TARGET_WEEKDAY = "三"  # 例：查星期三的課

# === 小工具函式 ===
TW_TZ = pytz.timezone(TIMEZONE)

week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
week_map_en = {"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

def normalize_weekday(w):
    if w in week_map_tw: return week_map_tw[w]
    if w in week_map_en: return week_map_en[w]
    raise ValueError("星期請用：一~日 或 Mon~Sun")

def ensure_datetime(s):
    try:
        return pd.to_datetime(s).date()
    except Exception:
        return pd.NaT

def week_monday(any_date):
    # 回到本週一
    return any_date - dt.timedelta(days=any_date.weekday())

def date_range_this_week(today=None):
    # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
    now = dt.datetime.now(TW_TZ).date() if today is None else today
    mon = week_monday(now)
    sun = mon + dt.timedelta(days=6)
    return mon, sun

def summarize_courses(day_df):
    if day_df.empty: return "本日無課"
    parts = []
    for _, r in day_df.iterrows():
        parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
    return "；".join(parts)

# === 過濾指定星期的課程（自動只抓「本週」） ===
df["日期"] = df["日期"].apply(ensure_datetime)
df = df.dropna(subset=["日期"])

target_wd = normalize_weekday(TARGET_WEEKDAY)
this_mon, this_sun = date_range_this_week()

mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])

# === 顯示查詢結果（指定星期的所有課） ===
display(Markdown(f"### 本週（{this_mon} ~ {this_sun}）星期{TARGET_WEEKDAY}課程"))
if today_df.empty:
    display(Markdown("> 本日無課"))
else:
    display(today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]])

# === 產生一句「行前提醒」 ===
def make_one_line_reminder(rows):
    # ── 方案A：不接模型的簡單規則版（免 API Key）
    if rows.empty:
        return "本日無課，不需準備。"
    items = []
    readings = []
    for _, r in rows.iterrows():
        if str(r.get("攜帶品","")).strip():
            items.append(str(r["攜帶品"]).strip())
        if str(r.get("先讀章節","")).strip():
            readings.append(str(r["先讀章節"]).strip())
    items_txt = "；".join(dict.fromkeys(items)) if items else "一般上課用品"
    read_txt  = "；".join(dict.fromkeys(readings)) if readings else "無指定預習"
    return f"請攜帶：{items_txt}；預習：{read_txt}。"

one_line = make_one_line_reminder(today_df)
display(Markdown("#### 行前提醒（一句話）"))
display(Markdown(f"> {one_line}"))

# === 回寫「本週重點」分頁 ===
# 若該分頁不存在就自動建立
SHEET_WEEKLY = "本週重點"
try:
    wk = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
except gspread.exceptions.WorksheetNotFound:
    wk = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

# 組成回寫資料：可以一週寫一列
summary_text = summarize_courses(df[mask_week].sort_values(["日期","時間(起)"]))
now_str = dt.datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")

row = {
    "週次起訖": f"{this_mon} ~ {this_sun}",
    "查詢星期": f"星期{TARGET_WEEKDAY}",
    "當日課程列表": summarize_courses(today_df),
    "行前提醒一句話": one_line,
    "更新時間": now_str
}

# 追加寫入（保留歷史）
existing = pd.DataFrame(wk.get_all_records())
outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True) if not existing.empty else pd.DataFrame([row])

wk.clear()
set_with_dataframe(wk, outdf)

display(Markdown("✅ 已回寫到 **本週重點** 分頁（保留歷史列）。"))

def load_gsheet_to_df():
    """
    從指定 Google Sheet (SHEET_URL + WORKSHEET_NAME)
    匯入資料到 pandas DataFrame，並回傳 df。
    """

    # --- 1. 從 gsheets 載入資料 ---
    print("📥 正在從 Google Sheet 匯入資料...")
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()

        if not sheets or len(sheets) < 2:
            print("⚠️ 該分頁沒有資料（只有標題或空白）。")
            return pd.DataFrame()

        # --- 2. 載入至 DataFrame ---
        df = pd.DataFrame(sheets[1:], columns=sheets[0])
        print(f"✅ 成功匯入 {len(df)} 筆資料。")

        # 預覽前 5 筆
        display(df.head())
        return df

    except gspread.exceptions.WorksheetNotFound:
        print(f"❌ 找不到分頁 '{WORKSHEET_NAME}'，請確認名稱是否正確。")
        return pd.DataFrame()

    except Exception as e:
        print(f"❌ 匯入 Google Sheet 時發生錯誤：{e}")
        return pd.DataFrame()

### 本週（2025-11-03 ~ 2025-11-09）星期三課程

,日期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
93,2025-11-05,試算表進階應用與程式設計,10:20,12:10,教402,,,
94,2025-11-05,體育(體操遊戲初級),13:20,15:10,體操房,,,


#### 行前提醒（一句話）

> 請攜帶：一般上課用品；預習：無指定預習。

✅ 已回寫到 **本週重點** 分頁（保留歷史列）。

In [ ]:
# === 加入 Gradio 編輯面板 ===
import gradio as gr

def save_edited_reminder(edited_text):
    """讓使用者編輯後回存到 Google Sheet"""
    from datetime import datetime
    import pytz
    tz = pytz.timezone(TIMEZONE)
    now_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M")

    # 重新讀取目前的分頁資料
    existing = pd.DataFrame(wk.get_all_records())
    # 追加更新列
    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{TARGET_WEEKDAY}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": edited_text,
        "更新時間": now_str
    }
    outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True)
    wk.clear()
    set_with_dataframe(wk, outdf)
    return f"✅ 已更新至 Google Sheet『本週重點』分頁，目前共 {len(outdf)} 筆。"

# Gradio 介面
with gr.Blocks(title="行前提醒編輯器") as demo:
    gr.Markdown("## ✏️ 行前提醒（可編輯並回存至 Google Sheet）")
    gr.Markdown("這裡會顯示模型或程式預設生成的行前提醒，使用者可以修改後按下『提交』回存。")
    reminder_box = gr.Textbox(label="行前提醒一句話", value=one_line, lines=3)
    save_btn = gr.Button("💾 提交並回存")
    status = gr.Markdown()

    save_btn.click(save_edited_reminder, inputs=[reminder_box], outputs=[status])

demo.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e43be46754c53d3f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import time
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
from google.colab import files
import os

from google.colab import userdata
try:
    api_key = userdata.get('gemini')
    print("✅ 已從 Colab Secrets 讀取 Gemini API Key。")
except Exception as e:
    print("⚠️ 無法從 Colab Secrets 讀取，請手動輸入：")
    api_key = input("請貼上你的 Gemini API Key：").strip()

os.environ["GEMINI_API_KEY"] = api_key
import google.generativeai as genai
genai.configure(api_key=api_key)
import pytz
from IPython.display import display, Markdown
import datetime as dt
import gradio as gr
import pandas as pd

def process_pdf_with_gemini(pdf_file):
    """
    處理使用者上傳的 PDF 檔案，支援 gradio.utils.NamedString、dict、file-like、path 字串或 bytes。
    1) 讀取上傳檔案
    2) 將檔案暫存到 /tmp
    3) 上傳到 Gemini File API，呼叫模型產生 CSV
    4) 刪除 Gemini 雲端暫存檔、清理本地暫存檔
    5) 將 Gemini 輸出寫成 gemini_schedule.csv
    回傳：字串狀態訊息（可直接顯示在 Gradio UI）
    重要：假設外部已經執行 genai.configure(api_key=...) 並完成 Google 認證 (gc 已存在)
    """

    try:
        # --- 1) 解析上傳物件，取得檔名與 bytes ---
        filename = None
        file_bytes = None

        # Gradio 可能回傳 dict-like（舊版/新版差異）
        if isinstance(pdf_file, dict):
            # 常見 key: 'name','data','tmp_path'
            if "data" in pdf_file and isinstance(pdf_file["data"], (bytes, bytearray)):
                filename = pdf_file.get("name", "upload.pdf")
                file_bytes = pdf_file["data"]
            elif "tmp_path" in pdf_file:
                tmp_path = pdf_file["tmp_path"]
                filename = os.path.basename(tmp_path)
                with open(tmp_path, "rb") as _f:
                    file_bytes = _f.read()
            else:
                # fallback: try 'name' as path
                namev = pdf_file.get("name")
                if isinstance(namev, str) and os.path.exists(namev):
                    filename = os.path.basename(namev)
                    with open(namev, "rb") as _f:
                        file_bytes = _f.read()

        # file-like (has .read)
        if file_bytes is None and hasattr(pdf_file, "read"):
            try:
                filename = getattr(pdf_file, "name", "upload.pdf")
                filename = os.path.basename(filename)
                try:
                    pdf_file.seek(0)
                except Exception:
                    pass
                file_bytes = pdf_file.read()
                if isinstance(file_bytes, str):
                    file_bytes = file_bytes.encode("utf-8")
            except Exception:
                file_bytes = None

        # Gradio NamedString: 沒有 read，但有 .name（為暫存路徑）
        if file_bytes is None and hasattr(pdf_file, "name") and not hasattr(pdf_file, "read"):
            path = pdf_file.name
            if isinstance(path, str) and os.path.exists(path):
                filename = os.path.basename(path)
                with open(path, "rb") as _f:
                    file_bytes = _f.read()

        # bytes 直接傳入
        if file_bytes is None and isinstance(pdf_file, (bytes, bytearray)):
            file_bytes = bytes(pdf_file)
            filename = "upload.pdf"

        # 字串路徑
        if file_bytes is None and isinstance(pdf_file, str):
            if os.path.exists(pdf_file):
                filename = os.path.basename(pdf_file)
                with open(pdf_file, "rb") as _f:
                    file_bytes = _f.read()

        if file_bytes is None or filename is None:
            return "❌ 無法解析上傳的檔案物件（型態不支援），請重新上傳或嘗試不同瀏覽器。"

        # --- 2) 檢查副檔名並寫入本地暫存檔（Gemini upload 需要 path） ---
        if not filename.lower().endswith(".pdf"):
            return f"⚠️ 警告：您上傳的檔案 '{filename}' 不是 PDF 檔。"

        temp_path = os.path.join("/tmp", f"{uuid.uuid4().hex}_{filename}")
        try:
            with open(temp_path, "wb") as tf:
                tf.write(file_bytes)
        except Exception as e:
            return f"❌ 無法將上傳檔案寫入暫存路徑：{e}"

        print(f"✅ 已儲存暫存檔：{temp_path} (大小 {len(file_bytes)} bytes)")
        print(f"準備處理檔案: '{filename}'")

        # --- 3) 上傳到 Gemini File API ---
        try:
            print("正在上傳 PDF 至 Gemini File API...")
            gemini_file = genai.upload_file(path=temp_path, display_name="114-1 課程表 PDF")
            print(f"✅ 檔案上傳成功！(File URI: {getattr(gemini_file, 'uri', 'unknown')})")
        except Exception as e:
            # 清理本地暫存
            try: os.remove(temp_path)
            except Exception: pass
            return f"❌ 上傳至 Gemini 失敗：{e}"

        # --- 4) 準備 Prompt & 呼叫模型 ---
        prompt_text = f"""
        你是一位 helpful AI assistant。
        請分析我上傳的 PDF 課表檔案 (檔案名稱: {filename})。

        這份 PDF 課表是「國立臺灣師範大學114學年度第1學期課表」。
        授課教師是「蔡芸琤」。

        請幫我將這份課表，轉換成每周的紀錄。
        學期開學日是 2025/9/1，學期結束日是 2025/12/19。

        你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，
        不要有任何其他說明文字 (例如 "好的，這是..." 或 "```csv" 標記)。

        CSV 格式的欄位必須如下：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"

        - 「攜帶品」、「先讀章節」、「備註」這三欄因為 PDF 中沒有，請保持空白。
        - 「日期」範圍請涵蓋 2025/9/1 到 2025/12/19 之間所有有課的日期。
        - 請仔細解析 PDF 中的表格，找出所有課程、時間和地點。
        """

        try:
            print("正在向 Gemini 2.5 Pro 提出請求 (可能需要一點時間)...")
            model = genai.GenerativeModel('gemini-2.5-pro')
            response = model.generate_content([prompt_text, gemini_file])
        except Exception as e:
            # 嘗試刪除遠端檔案再回傳錯誤
            try:
                if 'gemini_file' in locals():
                    genai.delete_file(gemini_file.name)
            except Exception:
                pass
            try: os.remove(temp_path)
            except Exception: pass
            return f"❌ 呼叫 Gemini 失敗：{e}"

        # --- 5) 刪除雲端暫存檔（盡量嘗試） ---
        try:
            genai.delete_file(gemini_file.name)
            print("🗑️ 已刪除 Gemini 雲端暫存檔。")
        except Exception:
            print("⚠️ 嘗試刪除 Gemini 雲端暫存檔失敗（忽略）")

        # --- 6) 解析模型回傳並寫成 CSV ---
        try:
            csv_output = response.text.strip().replace("```csv\n", "").replace("```", "").strip()
        except Exception as e:
            try: os.remove(temp_path)
            except Exception: pass
            return f"❌ 無法取得 Gemini 回傳的文字：{e}"

        try:
            with open("gemini_schedule.csv", "w", encoding="utf-8") as wf:
                wf.write(csv_output)
        except Exception as e:
            try: os.remove(temp_path)
            except Exception: pass
            return f"❌ 無法將 Gemini 輸出存成 gemini_schedule.csv：{e}"

        # --- 7) 清理本地暫存檔 ---
        try:
            os.remove(temp_path)
        except Exception:
            pass

        print("\n--- 🤖 Gemini 產生的 CSV 內容（前 400 字）---")
        print(csv_output[:400])
        print("---------------------------------")
        print("\n✅ 已將 Gemini 產生的結果儲存為 'gemini_schedule.csv'")

        return f"✅ PDF 解析成功，結果已儲存為 'gemini_schedule.csv'（共 {len(csv_output)} 字元）"

    except Exception as e:
        # 最後的總錯誤處理：嘗試刪除遠端檔並回報
        try:
            if 'gemini_file' in locals():
                genai.delete_file(gemini_file.name)
        except Exception:
            pass
        try:
            if 'temp_path' in locals() and os.path.exists(temp_path):
                os.remove(temp_path)
        except Exception:
            pass
        return f"❌ 未預期錯誤：{e}"


def upload_csv_to_gsheet():
    """
    將 Gemini 產生的 CSV ('gemini_schedule.csv') 上傳至指定的 Google Sheet。
    保留原有邏輯與輸出，只改成函式型態。
    """

    # --- 1. 定義您的 Google Sheet 資訊 (您已提供) ---
    SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit?gid=0#gid=0"
    WORKSHEET_NAME = "工作表1"
    TIMEZONE = "Asia/Taipei"

    try:
        # --- 2. 讀取 Gemini 產生的 CSV 檔案 ---
        df_to_upload = pd.read_csv("gemini_schedule.csv")
        print(f"✅ 成功讀取 'gemini_schedule.csv'，共 {len(df_to_upload)} 筆資料。")

        # --- 3. 準備 gspread (假設 gc 已經過授權) ---
        df_to_upload = df_to_upload.fillna('')

        # --- 4. 開啟 Google Sheet 和指定的分頁 ---
        print(f"正在開啟 Google Sheet (URL: {SHEET_URL})...")
        gsheets = gc.open_by_url(SHEET_URL)

        print(f"正在存取分頁: '{WORKSHEET_NAME}'...")
        try:
            worksheet = gsheets.worksheet(WORKSHEET_NAME)
        except gspread.exceptions.WorksheetNotFound:
            print(f"找不到分頁 '{WORKSHEET_NAME}'，將自動建立新分頁...")
            worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

        # --- 5. 將 DataFrame 轉換為 list (包含標頭) ---
        data_to_write = [df_to_upload.columns.values.tolist()] + df_to_upload.values.tolist()

        # --- 6. 清空分頁並寫入新資料 ---
        print("正在清空分頁內容...")
        worksheet.clear()

        print(f"正在將 {len(data_to_write) - 1} 筆資料寫入 '{WORKSHEET_NAME}' 分頁...")
        worksheet.update(data_to_write)

        print(f"\n✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '{WORKSHEET_NAME}' 分頁！")
        return "✅ 成功上傳至 Google Sheet！"

    except FileNotFoundError:
        msg = "❌ 錯誤：找不到 'gemini_schedule.csv' 檔案。請確認您已執行 Gemini API。"
        print(msg)
        return msg

    except pd.errors.EmptyDataError:
        msg = "❌ 錯誤：'gemini_schedule.csv' 是空的，沒有資料可以上傳。"
        print(msg)
        return msg

    except Exception as e:
        msg = f"❌ 執行時發生錯誤：{e}"
        print(msg)
        return msg

def analyze_weekly_schedule(df, TARGET_WEEKDAY="三"):
    """
    從 Google Sheet 匯入的課表 DataFrame (df)
    查詢本週指定星期的課程，顯示課表、行前提醒，
    並自動回寫至 Google Sheet 的「本週重點」分頁。
    """

    import pytz
    from IPython.display import display, Markdown
    import datetime as dt

    # === 參數：指定要查的星期 ===
    # 可填 "一","二","三","四","五","六","日"，或用英文 "Mon" ... "Sun"

    # === 小工具函式 ===
    TW_TZ = pytz.timezone(TIMEZONE)

    week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
    week_map_en = {"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

    def normalize_weekday(w):
        if w in week_map_tw: return week_map_tw[w]
        if w in week_map_en: return week_map_en[w]
        raise ValueError("星期請用：一~日 或 Mon~Sun")

    def ensure_datetime(s):
        try:
            return pd.to_datetime(s).date()
        except Exception:
            return pd.NaT

    def week_monday(any_date):
        # 回到本週一
        return any_date - dt.timedelta(days=any_date.weekday())

    def date_range_this_week(today=None):
        # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
        now = dt.datetime.now(TW_TZ).date() if today is None else today
        mon = week_monday(now)
        sun = mon + dt.timedelta(days=6)
        return mon, sun

    def summarize_courses(day_df):
        if day_df.empty: return "本日無課"
        parts = []
        for _, r in day_df.iterrows():
            parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
        return "；".join(parts)

    # === 過濾指定星期的課程（自動只抓「本週」） ===
    df["日期"] = df["日期"].apply(ensure_datetime)
    df = df.dropna(subset=["日期"])

    target_wd = normalize_weekday(TARGET_WEEKDAY)
    this_mon, this_sun = date_range_this_week()

    mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
    mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
    today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])

    # === 顯示查詢結果（指定星期的所有課） ===
    display(Markdown(f"### 本週（{this_mon} ~ {this_sun}）星期{TARGET_WEEKDAY}課程"))
    if today_df.empty:
        display(Markdown("> 本日無課"))
    else:
        display(today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]])

    # === 產生一句「行前提醒」 ===
    def make_one_line_reminder(rows):
        # ── 方案A：不接模型的簡單規則版（免 API Key）
        if rows.empty:
            return "本日無課，不需準備。"
        items = []
        readings = []
        for _, r in rows.iterrows():
            if str(r.get("攜帶品","")).strip():
                items.append(str(r["攜帶品"]).strip())
            if str(r.get("先讀章節","")).strip():
                readings.append(str(r["先讀章節"]).strip())
        items_txt = "；".join(dict.fromkeys(items)) if items else "一般上課用品"
        read_txt  = "；".join(dict.fromkeys(readings)) if readings else "無指定預習"
        return f"請攜帶：{items_txt}；預習：{read_txt}。"

    one_line = make_one_line_reminder(today_df)
    display(Markdown("#### 行前提醒（一句話）"))
    display(Markdown(f"> {one_line}"))

    # === 回寫「本週重點」分頁 ===
    # 若該分頁不存在就自動建立
    SHEET_WEEKLY = "本週重點"
    try:
        wk = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
    except gspread.exceptions.WorksheetNotFound:
        wk = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

    # 組成回寫資料：可以一週寫一列
    summary_text = summarize_courses(df[mask_week].sort_values(["日期","時間(起)"]))
    now_str = dt.datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")

    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{TARGET_WEEKDAY}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": one_line,
        "更新時間": now_str
    }

    # 追加寫入（保留歷史）
    existing = pd.DataFrame(wk.get_all_records())
    outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True) if not existing.empty else pd.DataFrame([row])

    wk.clear()
    set_with_dataframe(wk, outdf)

    display(Markdown("✅ 已回寫到 **本週重點** 分頁（保留歷史列）。"))

    return today_df, one_line


    # === Gradio 主流程 ===
def main_app():
    """
    建立 Gradio 介面：
    1️⃣ 上傳 PDF → 呼叫 process_pdf_with_gemini()
    2️⃣ 一鍵上傳至 Google Sheet → 呼叫 upload_csv_to_gsheet()
    3️⃣ 查詢課表 → 呼叫 analyze_weekly_schedule()
    """

    def step1_upload_pdf(pdf_file):
        return process_pdf_with_gemini(pdf_file)

    def step2_upload_to_gsheet():
        return upload_csv_to_gsheet()

    def step3_analyze_schedule(target_weekday):
        try:
            df = pd.read_csv("gemini_schedule.csv")
        except Exception as e:
            return f"❌ 無法讀取 gemini_schedule.csv：{e}", None

        _, one_line = analyze_weekly_schedule(df, TARGET_WEEKDAY=target_weekday)
        return f"✅ 查詢完成！行前提醒：{one_line}", one_line

    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 114-1 課表自動分析系統（Gemini + Google Sheet）")

        # --- Step 1 ---
        with gr.Tab("① 上傳 PDF 並用 Gemini 解析"):
            gr.Markdown("上傳課表 PDF，Gemini 會自動解析並轉成 CSV。")
            pdf_input = gr.File(label="上傳 PDF 檔案")
            pdf_output = gr.Textbox(label="處理結果")
            pdf_btn = gr.Button("🚀 開始解析 PDF")
            pdf_btn.click(step1_upload_pdf, inputs=pdf_input, outputs=pdf_output)

        # --- Step 2 ---
        with gr.Tab("② 上傳 CSV 至 Google Sheet"):
            gr.Markdown("將 Gemini 產生的 CSV 上傳至 Google Sheet。")
            gsheet_output = gr.Textbox(label="上傳結果")
            gsheet_btn = gr.Button("☁️ 上傳至 Google Sheet")
            gsheet_btn.click(step2_upload_to_gsheet, outputs=gsheet_output)

        # --- Step 3 ---
        with gr.Tab("③ 查詢本週課程"):
            gr.Markdown("輸入想查詢的星期（例：「一」「二」「三」或 Mon/Tue/Wed...）")
            weekday_input = gr.Textbox(label="查詢星期", value="三")
            analyze_output = gr.Textbox(label="查詢結果")
            analyze_btn = gr.Button("🔍 查詢本週課程")
            analyze_btn.click(step3_analyze_schedule, inputs=weekday_input, outputs=analyze_output)

        gr.Markdown("---")
        gr.Markdown("👩‍🏫 開發者：ChatGPT 自動整合（Gemini + GSheet + Gradio）")

    return demo


# === 啟動 Gradio App ===
demo = main_app()
demo.launch()

⚠️ 無法從 Colab Secrets 讀取，請手動輸入：
請貼上你的 Gemini API Key：*******
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abe9253e700628fb33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --upgrade gradio google-generativeai
# ===========================
#      🍀 Imports & Setup
# ===========================
import os
import uuid
import pandas as pd
import gradio as gr
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth, userdata
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe

# --- Google 認證 ---
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===========================
#      🔑 Gemini API Key
# ===========================
try:
    api_key = userdata.get("gemini")
    print("✅ 已從 Colab Secrets 讀取 Gemini API Key")
except Exception:
    print("⚠️ 無法從 Secrets 讀取，請手動輸入：")
    api_key = input("請輸入 Gemini API Key：").strip()

genai.configure(api_key=api_key)

# ===========================
#      📄 Step 1: PDF → CSV
#      同步版（Colab friendly）
# ===========================
import asyncio

def process_pdf_with_gemini_sync(pdf_file):
    """同步包裝 async 版，避免 Colab loop 衝突"""
    return asyncio.run(_process_pdf_with_gemini_async(pdf_file))

async def _process_pdf_with_gemini_async(pdf_file):
    import time
    import google.generativeai as genai

    try:
        # 1. 取得路徑
        if hasattr(pdf_file, "name"):
            pdf_path = pdf_file.name
        else:
            pdf_path = pdf_file

        print(f"📄 [DEBUG] 檔案路徑：{pdf_path}")

        # 2. 上傳 PDF
        print("⏫ [DEBUG] 正在上傳 PDF 到 Gemini File API ...")
        gemini_file = await genai.upload_file(pdf_path)
        print(gemini_file.__dict__)  # 檢查是否有正確的 name/id
        print(f"✅ 上傳成功！File ID = {gemini_file.name}")

        # 3. 等待 AI 處理 PDF
        print("⏳ [DEBUG] 等待後端處理 PDF ...")
        while True:
            f = await genai.get_file(gemini_file.name)
            print(f"    ➤ 狀態：{f.state}")

            if f.state == "PROCESSING":
                print("    ... PDF 後端仍在解析中（可能要 20–180 秒）")
                await asyncio.sleep(5)
            elif f.state == "FAILED":
                return "❌ PDF 處理失敗（Gemini 無法解析此 PDF）"
            elif f.state == "READY":
                print("🎉 PDF 已準備好！開始丟給模型解析...")
                break
            else:
                print("    不明狀態，繼續等...")
                await asyncio.sleep(5)

        # 4. 準備 prompt
        prompt_text = """
        請將 PDF 課表轉成 CSV，欄位：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
        僅輸出 CSV，不要額外文字。
        """

        # 5. 呼叫模型
        model = genai.GenerativeModel("gemini-2.0-pro-exp")
        print("🤖 [DEBUG] 開始呼叫模型...")
        response = await model.generate_content([prompt_text, gemini_file])

        print("✨ [DEBUG] 模型已回應（一部分預覽）")
        print(response.text[:300])

        return response.text

    except Exception as e:
        return f"❌ 發生錯誤：{e}"


# ===========================
#   ☁️ Step 2: CSV → GSheet
# ===========================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"

def upload_csv_to_gsheet():
    """將 CSV 上傳至 Google Sheet"""

    try:
        df = pd.read_csv("gemini_schedule.csv").fillna("")
    except:
        return "❌ 找不到 CSV，請先解析 PDF"

    try:
        gs = gc.open_by_url(SHEET_URL)
        try:
            ws = gs.worksheet(WORKSHEET_NAME)
        except:
            ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    except Exception as e:
        return f"❌ Google Sheet 開啟失敗：{e}"

    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())

    return "✅ 已成功寫入 Google Sheet！"


# ===========================
#     🔍 Step 3: 查詢課表
# ===========================
import datetime as dt
import pytz
TIMEZONE = "Asia/Taipei"

def analyze_weekly_schedule(weekday):
    """查詢本週某星期的課程"""

    try:
        df = pd.read_csv("gemini_schedule.csv")
    except:
        return "❌ 請先完成 PDF 解析和 CSV 產生"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")

    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    # 轉換星期
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
    if weekday not in week_map:
        return "❌ 请输入「一」「二」…「日」"

    target = week_map[weekday]

    mask = (df["日期"].dt.date >= week_mon) & \
           (df["日期"].dt.date <= week_sun) & \
           (df["日期"].dt.dayofweek == target)

    today_df = df[mask].sort_values("時間(起)")

    # 行前提醒
    if today_df.empty:
        reminder = "本日無課，不需準備。"
    else:
        reminder = "；".join(
            f"{r['課程名稱']} ({r['時間(起)']}~{r['時間(迄)']})"
            for _, r in today_df.iterrows()
        )

    return f"📅 本週星期{weekday} 課表：\n{today_df.to_string(index=False)}\n\n🎒 行前提醒：{reminder}"


# ===========================
#         🎛️ Gradio UI
# ===========================
def main_app():

    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 114-1 課表自動分析（Gemini + GSheet）")

        # ---- Step 1 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_with_gemini_sync, pdf_input, out1)
        # ---- Step 2 ----
        with gr.Tab("② 上傳至 Google Sheet"):
            out2 = gr.Textbox(label="結果")
            gr.Button("上傳 CSV").click(upload_csv_to_gsheet, outputs=out2)

        # ---- Step 3 ----
        with gr.Tab("③ 查詢課表"):
            weekday = gr.Textbox(label="星期（例：三）", value="三")
            out3 = gr.Textbox(label="查詢結果")
            gr.Button("查詢").click(analyze_weekly_schedule, weekday, out3)

    return demo


demo = main_app()
demo.launch(share=True, inline=False, inbrowser=True, debug=True)


✅ 已從 Colab Secrets 讀取 Gemini API Key
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30f544e68c57606a77.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

📄 [DEBUG] 檔案路徑：/tmp/gradio/fe8520e26b4a4acd0cfa882a36ef736ed8789e7b9046457aa8aabb0e5bc92dd4/export.pdf
⏫ [DEBUG] 正在上傳 PDF 到 Gemini File API ...
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://30f544e68c57606a77.gradio.live


In [ ]:
!pip install --upgrade google-generativeai gspread gspread-dataframe pandas

import os
import pandas as pd
import google.generativeai as genai
from google.colab import auth, userdata
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import time

# -----------------------
# Google 認證
# -----------------------
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# -----------------------
# Gemini API Key
# -----------------------
try:
    api_key = userdata.get("gemini")
    print("✅ 已從 Colab Secrets 讀取 Gemini API Key")
except Exception:
    print("⚠️ 無法從 Secrets 讀取，請手動輸入：")
    api_key = input("請輸入 Gemini API Key：").strip()

genai.configure(api_key=api_key)

# -----------------------
# PDF -> CSV
# -----------------------
def process_pdf_with_gemini(pdf_file_path):
    try:
        print(f"📄 [DEBUG] 檔案路徑：{pdf_file_path}")

        # 上傳 PDF
        print("⏫ [DEBUG] 正在上傳 PDF 到 Gemini File API ...")
        gemini_file = genai.upload_file(pdf_file_path)
        print(f"✅ 上傳成功！File ID = {gemini_file.name}")

        # 等待後端處理 PDF
        print("⏳ [DEBUG] 等待後端處理 PDF ...")
        while True:
            f = genai.get_file(gemini_file.name)
            print(f"    ➤ 狀態：{f.state}")
            if f.state == "PROCESSING":
                time.sleep(5)
            elif f.state == "FAILED":
                return "❌ PDF 處理失敗"
            elif f.state == "READY":
                print("🎉 PDF 已準備好！")
                break

        # 準備 prompt
        prompt_text = """
        請將 PDF 課表轉成 CSV，欄位：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
        僅輸出 CSV，不要額外文字。
        """

        model = genai.GenerativeModel("gemini-2.0-pro-exp")
        print("🤖 開始呼叫模型解析 PDF ...")
        response = model.generate_content([prompt_text, gemini_file])
        csv_text = response.text

        # 儲存 CSV
        csv_path = "gemini_schedule.csv"
        with open(csv_path, "w", encoding="utf-8") as f:
            f.write(csv_text)

        print(f"✅ CSV 已儲存：{csv_path}")
        return csv_path

    except Exception as e:
        return f"❌ 發生錯誤：{e}"

# -----------------------
# CSV -> Google Sheet
# -----------------------
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"

def upload_csv_to_gsheet(csv_path="gemini_schedule.csv"):
    try:
        df = pd.read_csv(csv_path).fillna("")
    except:
        return "❌ 找不到 CSV，請先解析 PDF"

    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")

    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())
    return "✅ 已成功寫入 Google Sheet！"

# -----------------------
# 查詢課表
# -----------------------
TIMEZONE = "Asia/Taipei"
def analyze_weekly_schedule(weekday, csv_path="gemini_schedule.csv"):
    try:
        df = pd.read_csv(csv_path)
    except:
        return "❌ 請先完成 PDF 解析和 CSV 產生"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
    if weekday not in week_map:
        return "❌ 请输入「一」「二」…「日」"

    target = week_map[weekday]
    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun) & \
           (df["日期"].dt.dayofweek == target)
    today_df = df[mask].sort_values("時間(起)")

    if today_df.empty:
        reminder = "本日無課，不需準備。"
    else:
        reminder = "；".join(f"{r['課程名稱']} ({r['時間(起)']}~{r['時間(迄)']})" for _, r in today_df.iterrows())

    return f"📅 本週星期{weekday} 課表：\n{today_df.to_string(index=False)}\n\n🎒 行前提醒：{reminder}"

# -----------------------
# === 使用示例 ===
# -----------------------
# 1️⃣ 上傳 PDF，解析成 CSV
pdf_path = "/content/你的課表.pdf"  # 改成你的 PDF 檔案路徑
csv_file = process_pdf_with_gemini(pdf_path)

# 2️⃣ 上傳 CSV 到 Google Sheet
print(upload_csv_to_gsheet(csv_file))

# 3️⃣ 查詢課表
print(analyze_weekly_schedule("三", csv_file))


✅ 已從 Colab Secrets 讀取 Gemini API Key
📄 [DEBUG] 檔案路徑：/content/你的課表.pdf
⏫ [DEBUG] 正在上傳 PDF 到 Gemini File API ...
❌ 找不到 CSV，請先解析 PDF
❌ 請先完成 PDF 解析和 CSV 產生


In [ ]:
from google.colab import files

uploaded = files.upload()  # 會跳出選擇檔案的視窗
gemini_file = genai.upload_file("export.pdf")
print(gemini_file.__dict__)

Saving export.pdf to export (1).pdf
{'_proto': name: "files/6v8wvnorwqhq"
display_name: "export.pdf"
mime_type: "application/pdf"
size_bytes: 57783
create_time {
  seconds: 1763272151
  nanos: 803409000
}
update_time {
  seconds: 1763272151
  nanos: 803409000
}
expiration_time {
  seconds: 1763444951
  nanos: 642890758
}
sha256_hash: "56787615bc7b4de715e047efed47210f4d2f59094637dab0afe03300d70b8e1e"
uri: "https://generativelanguage.googleapis.com/v1beta/files/6v8wvnorwqhq"
state: ACTIVE
}


In [ ]:
#!pip install --upgrade gspread gspread-dataframe pandas gradio

import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    # 這邊保留原本功能
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def query_schedule(weekday):
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
    if weekday not in week_map:
        return "❌ 請輸入「一」「二」…「日」"
    target = week_map[weekday]

    mask = (df["日期"].dt.date >= week_mon) & \
           (df["日期"].dt.date <= week_sun) & \
           (df["日期"].dt.dayofweek == target)
    today_df = df[mask].sort_values("時間(起)")

    if today_df.empty:
        reminder = "本日無課，不需準備。"
    else:
        reminder = "；".join(
            f"{r['課程名稱']} ({r['時間(起)']}~{r['時間(迄)']})"
            for _, r in today_df.iterrows()
        )
    return f"📅 本週星期{weekday} 課表：\n{today_df.to_string(index=False)}\n\n🎒 行前提醒：{reminder}"

# ===============================
# Step 3: 改寫課程
# ===============================
def edit_course(course_name, carry_item, pre_read, note):
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return "❌ Google Sheet 讀取失敗"

    # 找到課程
    mask = df["課程名稱"] == course_name
    if not mask.any():
        return f"❌ 找不到課程 {course_name}"

    df.loc[mask, "攜帶品"] = carry_item
    df.loc[mask, "先讀章節"] = pre_read
    df.loc[mask, "備註"] = note

    # 更新回 Google Sheet
    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())
    return f"✅ 課程 {course_name} 已更新！"

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            weekday = gr.Textbox(label="星期（例：三）", value="三")
            out2 = gr.Textbox(label="查詢結果")
            gr.Button("查詢").click(query_schedule, weekday, out2)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            course_name = gr.Textbox(label="課程名稱")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")
            gr.Button("更新課程").click(edit_course,
                                         [course_name, carry_item, pre_read, note],
                                         out3)
    return demo

demo = main_app()
demo.launch(share=True, inline=False, inbrowser=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5f8237dde34b39a0a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5f8237dde34b39a0a2.gradio.live


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 只顯示主要欄位
    display_df = week_df[["日期","星期","課程名稱","時間(起)","時間(迄)","地點"]]
    return display_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_courses_by_weekday(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return [""]
    day_index = week_map[weekday]

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    courses = df.loc[mask, "課程名稱"].tolist()
    return courses if courses else [""]

# 根據選擇課程載入原本資料
def load_course_info(course_name):
    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    mask = df["課程名稱"] == course_name
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_course(course_name, carry_item, pre_read, note):
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return "❌ Google Sheet 讀取失敗"

    mask = df["課程名稱"] == course_name
    if not mask.any():
        return f"❌ 找不到課程 {course_name}"

    df.loc[mask, "攜帶品"] = carry_item
    df.loc[mask, "先讀章節"] = pre_read
    df.loc[mask, "備註"] = note

    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())
    return f"✅ 課程 {course_name} 已更新！"

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            course_dropdown = gr.Dropdown(choices=[""], label="選擇課程")

            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 選星期時更新課程選單
            weekday_dropdown.change(fn=get_courses_by_weekday, inputs=weekday_dropdown, outputs=course_dropdown)

            # 選課程時讀取原本欄位
            course_dropdown.change(fn=lambda x: load_course_info(x) if x else ("", "", ""),
                                inputs=course_dropdown,
                                outputs=[carry_item, pre_read, note])

            # 更新課程
            update_btn = gr.Button("更新課程")
            update_btn.click(fn=edit_course, inputs=[course_dropdown, carry_item, pre_read, note], outputs=out3)

    return demo

demo = main_app()
demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a50aadc5f066e6a6b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 只顯示主要欄位
    display_df = week_df[["日期","星期","課程名稱","時間(起)","時間(迄)","地點"]]
    return display_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()  # 這裡拿完整欄位 df
    # 把每一行轉成字串
    options = week_df.apply(lambda row: f"{row['日期'].date()}\t{row['星期']}\t{row['課程名稱']}\t{row['時間(起)']}\t{row['時間(迄)']}\t{row['地點']}", axis=1).tolist()
    return options if options else [""]

def load_row_info(selected_row_str):
    if not selected_row_str:
        return "", "", ""

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    # 將選單字串拆回欄位，用來比對唯一行
    parts = selected_row_str.split("\t")
    date, weekday, course_name, start, end, location = parts

    mask = (
        (df["日期"] == date) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start) &
        (df["時間(迄)"] == end) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "", "", ""

    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_course_by_row(selected_row_str, carry_item, pre_read, note):
    if not selected_row_str:
        return "❌ 未選擇任何課程"

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    parts = selected_row_str.split("\t")
    date, weekday, course_name, start, end, location = parts

    mask = (
        (df["日期"] == date) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start) &
        (df["時間(迄)"] == end) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程"

    df.loc[mask, "攜帶品"] = carry_item
    df.loc[mask, "先讀章節"] = pre_read
    df.loc[mask, "備註"] = note

    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())
    return f"✅ 課程 {course_name} 已更新！"

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            week_rows_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out = gr.Textbox(label="更新結果")

            # 當選擇課程行時載入原本資料
            week_rows_dropdown.change(load_row_info, inputs=week_rows_dropdown, outputs=[carry_item, pre_read, note])

            # 更新課程
            gr.Button("更新課程").click(edit_course_by_row,
                                        [week_rows_dropdown, carry_item, pre_read, note],
                                        out)

    return demo

demo = main_app()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02f9a1a7e6d8839ff1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 只顯示主要欄位
    display_df = week_df[["日期","星期","課程名稱","時間(起)","時間(迄)","地點"]]
    return display_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return f"❌ 找不到課程 {course_name} on {date} {start_time}"

    df.loc[mask, "攜帶品"] = carry_item
    df.loc[mask, "先讀章節"] = pre_read
    df.loc[mask, "備註"] = note

    ws.clear()
    ws.update([df.columns.values.tolist()] + df.values.tolist())
    return f"✅ 課程 {course_name} 已更新！"

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

    return demo

demo = main_app()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://622fdc2270aed02dea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import io
import os

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 只顯示主要欄位
    display_df = week_df[["日期","星期","課程名稱","時間(起)","時間(迄)","地點"]]
    return display_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"

    parts = selected_row.split("\t")
    if len(parts) < 6:
        return "❌ 無法解析選擇的行"

    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    # 找到目標行
    mask = (
        (df["日期"] == pd.to_datetime(date).date()) &
        (df["星期"] == weekday) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start_time) &
        (df["時間(迄)"] == end_time) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程，請檢查 Google Sheet 資料是否格式一致"

    # Google Sheet 第幾列（+2 因為 index 從 0 且第 1 列是標題）
    row_index = df.index[mask][0] + 2

    # 更新 Google Sheet（安全版，只動該行欄位）
    ws.update_cell(row_index, df.columns.get_loc("攜帶品") + 1, carry_item)
    ws.update_cell(row_index, df.columns.get_loc("先讀章節") + 1, pre_read)
    ws.update_cell(row_index, df.columns.get_loc("備註") + 1, note)

    return "✅ 更新成功（你的資料不會被刪除）！"


# 匯出指定週次 CSV（目前先只實作「本週」）
def export_week_csv(week_option):
    week_df, _ = load_week_schedule()

    file_path = "/content/本週課表.csv"
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    return file_path

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

        with gr.Tab("④ 匯出 CSV"):
            week_dropdown = gr.Dropdown(choices=["本週"], label="選擇週次")
            download_btn = gr.File(label="下載 CSV")
            gr.Button("匯出 CSV").click(export_week_csv, inputs=week_dropdown, outputs=download_btn)

    return demo

demo = main_app()
demo.launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://348d87dc730b886f7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import io
import os

# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 保留所有欄位
    return week_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"

    parts = selected_row.split("\t")
    if len(parts) < 6:
        return "❌ 無法解析選擇的行"

    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    # 找到目標行
    mask = (
        (df["日期"] == pd.to_datetime(date).date()) &
        (df["星期"] == weekday) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start_time) &
        (df["時間(迄)"] == end_time) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程，請檢查 Google Sheet 資料是否格式一致"

    # Google Sheet 第幾列（+2 因為 index 從 0 且第 1 列是標題）
    row_index = df.index[mask][0] + 2

    # 更新 Google Sheet（安全版，只動該行欄位）
    ws.update_cell(row_index, df.columns.get_loc("攜帶品") + 1, carry_item)
    ws.update_cell(row_index, df.columns.get_loc("先讀章節") + 1, pre_read)
    ws.update_cell(row_index, df.columns.get_loc("備註") + 1, note)

    return "✅ 更新成功（你的資料不會被刪除）！"


# 匯出指定週次 CSV（目前先只實作「本週」）
def export_week_csv(week_option):
    week_df, _ = load_week_schedule()

    file_path = "/content/本週課表.csv"
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    return file_path

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

        with gr.Tab("④ 匯出 CSV"):
            week_dropdown = gr.Dropdown(choices=["本週"], label="選擇週次")
            download_btn = gr.File(label="下載 CSV")
            gr.Button("匯出 CSV").click(export_week_csv, inputs=week_dropdown, outputs=download_btn)

    return demo

demo = main_app()
demo.launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26b2f119c44fa36dc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import io
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 保留所有欄位
    return week_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"

    parts = selected_row.split("\t")
    if len(parts) < 6:
        return "❌ 無法解析選擇的行"

    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    # 找到目標行
    mask = (
        (df["日期"] == pd.to_datetime(date).date()) &
        (df["星期"] == weekday) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start_time) &
        (df["時間(迄)"] == end_time) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程，請檢查 Google Sheet 資料是否格式一致"

    # Google Sheet 第幾列（+2 因為 index 從 0 且第 1 列是標題）
    row_index = df.index[mask][0] + 2

    # 更新 Google Sheet（安全版，只動該行欄位）
    ws.update_cell(row_index, df.columns.get_loc("攜帶品") + 1, carry_item)
    ws.update_cell(row_index, df.columns.get_loc("先讀章節") + 1, pre_read)
    ws.update_cell(row_index, df.columns.get_loc("備註") + 1, note)

    return "✅ 更新成功（你的資料不會被刪除）！"


# 匯出指定週次 CSV（目前先只實作「本週」）
def export_week_csv(week_option):
    week_df, _ = load_week_schedule()

    file_path = "/content/本週課表.csv"
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    return file_path

def send_csv_email(file_path, recipient_email):
    """
    file_path: CSV 檔案完整路徑
    recipient_email: 收件人 Gmail
    """
    sender_email = "41371232h@gapps.ntnu.edu.tw"        # 改成你的 Gmail
    app_password = "s**************l"      # 必須申請 App Password

    # 建立郵件
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = "本週課表 CSV"

    # 郵件內文
    msg.attach(MIMEText("請查收本週課表 CSV", "plain"))

    # 附加 CSV
    with open(file_path, "rb") as f:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={file_path.split('/')[-1]}")
        msg.attach(part)

    # 發送
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)

    return "✅ CSV 已成功寄到 Gmail！"

# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

        with gr.Tab("④ 匯出 CSV"):
            week_dropdown = gr.Dropdown(choices=["本週"], label="選擇週次")
            download_btn = gr.File(label="下載 CSV")
            gr.Button("匯出 CSV").click(export_week_csv, inputs=week_dropdown, outputs=download_btn)

            # ---- 新增按鈕：寄送 CSV 到 Gmail ----
            recipient_input = gr.Textbox(label="收件人 Gmail", placeholder="你的郵件")
            send_btn = gr.Button("寄送 CSV 到 Gmail")
            send_out = gr.Textbox(label="寄送結果")

            # 寄送按鈕：輸入 CSV 路徑 + 收件人
            # 這裡固定 CSV 路徑和 export_week_csv 產生的檔案一致
            send_btn.click(
                fn=lambda recipient_email: send_csv_email("/content/本週課表.csv", recipient_email),
                inputs=recipient_input,
                outputs=send_out
            )

    return demo

demo = main_app()
demo.launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e52d9ab12f63e9e7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import gradio as gr
import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
import datetime as dt
import pytz
import io
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
# Google Auth
from google.colab import auth
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# ===============================
# Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/1OEB6dXPnqn6qdxXOv8I3Lv0t2oHL_zztVn8xnk6vpY8/edit"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

def get_worksheet():
    gs = gc.open_by_url(SHEET_URL)
    try:
        ws = gs.worksheet(WORKSHEET_NAME)
    except:
        ws = gs.add_worksheet(WORKSHEET_NAME, rows="100", cols="20")
    return ws

# ===============================
# Step 1: 解析 PDF (保留)
# ===============================
def process_pdf_dummy(pdf_file):
    return "PDF 解析功能保留（假設已成功）"

# ===============================
# Step 2: 查詢課表
# ===============================
def load_week_schedule():
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame(), "❌ Google Sheet 讀取失敗"

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    tw = pytz.timezone(TIMEZONE)
    today = dt.datetime.now(tw).date()
    week_mon = today - dt.timedelta(days=today.weekday())
    week_sun = week_mon + dt.timedelta(days=6)

    mask = (df["日期"].dt.date >= week_mon) & (df["日期"].dt.date <= week_sun)
    week_df = df[mask].sort_values(["日期", "時間(起)"])

    # 保留所有欄位
    return week_df, ""

# 查詢特定星期：顯示完整欄位（包含攜帶品、先讀章節、備註）
def query_day_schedule(weekday):
    week_map = {"一":0,"二":1,"三":2,"四":3,"五":4}
    if weekday not in week_map:
        return pd.DataFrame()
    day_index = week_map[weekday]

    # 重新讀取完整 Google Sheet 資料
    try:
        ws = get_worksheet()
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except:
        return pd.DataFrame()

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce")
    mask = df["日期"].dt.dayofweek == day_index
    day_df = df[mask].sort_values("時間(起)")

    return day_df  # 這裡回傳完整欄位

# ===============================
# Step 3: 改寫課程
# ===============================
def get_week_rows_for_edit():
    week_df, _ = load_week_schedule()
    # 顯示主要欄位作為選單文字
    rows = week_df.apply(lambda x: f'{x["日期"].date()}\t{x["星期"]}\t{x["課程名稱"]}\t{x["時間(起)"]}\t{x["時間(迄)"]}\t{x["地點"]}', axis=1)
    return rows.tolist() if not rows.empty else [""]

# 根據選擇的整行，載入原本資料
def load_row_info(selected_row):
    if not selected_row:
        return "", "", "", "", "", ""
    parts = selected_row.split("\t")
    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    mask = (df["日期"] == pd.to_datetime(date).date()) & (df["課程名稱"] == course_name) & (df["時間(起)"] == start_time)
    if not mask.any():
        return "", "", ""
    row = df.loc[mask].iloc[0]
    return str(row["攜帶品"]), str(row["先讀章節"]), str(row["備註"])

def edit_row(selected_row, carry_item, pre_read, note):
    if not selected_row:
        return "❌ 請選擇一行課程"

    parts = selected_row.split("\t")
    if len(parts) < 6:
        return "❌ 無法解析選擇的行"

    date, weekday, course_name, start_time, end_time, location = parts

    ws = get_worksheet()
    data = ws.get_all_records()
    df = pd.DataFrame(data)

    df["日期"] = pd.to_datetime(df["日期"], errors="coerce").dt.date

    # 找到目標行
    mask = (
        (df["日期"] == pd.to_datetime(date).date()) &
        (df["星期"] == weekday) &
        (df["課程名稱"] == course_name) &
        (df["時間(起)"] == start_time) &
        (df["時間(迄)"] == end_time) &
        (df["地點"] == location)
    )

    if not mask.any():
        return "❌ 找不到課程，請檢查 Google Sheet 資料是否格式一致"

    # Google Sheet 第幾列（+2 因為 index 從 0 且第 1 列是標題）
    row_index = df.index[mask][0] + 2

    # 更新 Google Sheet（安全版，只動該行欄位）
    ws.update_cell(row_index, df.columns.get_loc("攜帶品") + 1, carry_item)
    ws.update_cell(row_index, df.columns.get_loc("先讀章節") + 1, pre_read)
    ws.update_cell(row_index, df.columns.get_loc("備註") + 1, note)

    return "✅ 更新成功（你的資料不會被刪除）！"


# 匯出指定週次 CSV（目前先只實作「本週」）
def export_week_csv(week_option):
    week_df, _ = load_week_schedule()

    file_path = "/content/本週課表.csv"
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    return file_path

def export_and_send_week_csv(recipient_email):
    # 1️⃣ 先生成 CSV
    file_path = "/content/本週課表.csv"
    week_df, _ = load_week_schedule()
    week_df.to_csv(file_path, index=False, encoding="utf-8-sig")  # utf-8-sig 方便 Excel 打開

    # 2️⃣ 寄信
    sender_email = "41371232h@gapps.ntnu.edu.tw"
    app_password = "s**************l"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = "本週課表 CSV"
    msg.attach(MIMEText("請查收本週課表 CSV", "plain"))

    with open(file_path, "rb") as f:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(f.read())
        encoders.encode_base64(part)

        # 中文檔名正確
        from email.utils import encode_rfc2231
        filename = "本週課表.csv"
        filename_encoded = encode_rfc2231(filename, 'utf-8')
        part.add_header(
            "Content-Disposition",
            f"attachment; filename*=UTF-8''{filename_encoded}"
        )
        msg.attach(part)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)

    return "✅ CSV 已成功寄到 Gmail！"
# ===============================
# Gradio UI
# ===============================
def main_app():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🎓 NTNU 課表管理系統")

        # ---- Tab 1: PDF 解析 ----
        with gr.Tab("① 解析 PDF"):
            pdf_input = gr.File(label="上傳 PDF 檔案")
            out1 = gr.Textbox(label="結果")
            gr.Button("開始解析").click(process_pdf_dummy, pdf_input, out1)

        # ---- Tab 2: 查詢課表 ----
        with gr.Tab("② 查詢課表"):
            week_df, _ = load_week_schedule()
            df_display = gr.Dataframe(value=week_df, label="本週課表", interactive=False)

            weekday_dropdown = gr.Dropdown(choices=["一","二","三","四","五"], label="選擇星期")
            day_df = gr.Dataframe(value=pd.DataFrame(), label="該日課表", interactive=False)
            gr.Button("查詢").click(query_day_schedule, weekday_dropdown, day_df)

        # ---- Tab 3: 改寫課程 ----
        with gr.Tab("③ 改寫課程"):
            row_dropdown = gr.Dropdown(choices=get_week_rows_for_edit(), label="選擇課程行")
            carry_item = gr.Textbox(label="攜帶品")
            pre_read = gr.Textbox(label="先讀章節")
            note = gr.Textbox(label="備註")
            out3 = gr.Textbox(label="更新結果")

            # 當選擇行時載入原本資料
            row_dropdown.change(fn=load_row_info, inputs=row_dropdown, outputs=[carry_item, pre_read, note])
            # 更新課程
            gr.Button("更新課程").click(edit_row, [row_dropdown, carry_item, pre_read, note], out3)

        with gr.Tab("④ 匯出 CSV"):
            week_dropdown = gr.Dropdown(choices=["本週"], label="選擇週次")
            download_btn = gr.File(label="下載 CSV")
            gr.Button("匯出 CSV").click(export_week_csv, inputs=week_dropdown, outputs=download_btn)

            # ---- 新增按鈕：寄送 CSV 到 Gmail ----
            recipient_input = gr.Textbox(label="收件人 Gmail", placeholder="你的郵件")
            send_btn = gr.Button("寄送 CSV 到 Gmail")
            send_out = gr.Textbox(label="寄送結果")

            # 寄送按鈕：輸入 CSV 路徑 + 收件人
            # 這裡固定 CSV 路徑和 export_week_csv 產生的檔案一致
            send_btn.click(
                fn=export_and_send_week_csv,
                inputs=recipient_input,
                outputs=send_out
            )

    return demo

demo = main_app()
demo.launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://17254aca0223136523.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
